# Word Embeddings : le modèle Word2Vec

## Imports

In [42]:
import sys
import os
import re
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

### telecharger les stops words

In [43]:
# Télécharger stopwords
nltk.download('stopwords')
sw = set(stopwords.words("french"))

# Stopwords supplémentaires
extra_sw = {
    "les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
    "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
    "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
    "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
    "van", "het", "autre", "jusqu", "ville", "rossel", "dem", "etxc", "elles", "dés",
    "prixx", "écr", "géné", "app", "adr", "mod", "bur", "trav", "et", "de", "à", "en",
    "le", "la", "du", "des", "aux", "un", "une"
}
sw |= extra_sw

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilaria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Fonction nettoyage de texte

In [45]:
def clean_text_string(text):
    # Supprime tout ce qui n’est pas une lettre (remplace par espace)
    text = re.sub(r"[^a-zA-ZÀ-ÿ\s]", " ", text)
    # Minuscules
    text = text.lower()
    # Tokenisation simple
    words = nltk.wordpunct_tokenize(text)
    # Filtrer mots alphabétiques, >2 lettres, non-stopwords
    kept = [w for w in words if len(w) > 2 and w.isalpha() and w not in sw]
    return " ".join(kept)


### Lecture du texte nettoyé

In [47]:
# Chemin vers ton fichier
infile = "../../data/sents.txt"

# Lecture de toutes les lignes
with open(infile, encoding="utf-8") as f:
    texts = f.readlines()

# Nettoyage
cleaned_texts = [clean_text_string(text) for text in texts]
# On enlève les lignes vides après nettoyage
cleaned_texts = [t for t in cleaned_texts if t.strip() != ""]

print(f"Nombre de phrases nettoyées : {len(cleaned_texts)}")
print(cleaned_texts[:5])  # afficher les 5 premières phrases pour vérification


Nombre de phrases nettoyées : 891927
['imnri hmu marché tenu hors villa', 'vaain téicj races indigènes', 'rasa îichakdui taureaux iallsènas', 'hollandais dufr', 'îdto vachei laitières vante']


## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [48]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [49]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

In [50]:
import os
print(os.getcwd())


/Users/ilaria/Desktop/STIC/traitement_auto_corpus/tac/tps/tp3


### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [51]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [52]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [53]:
len(bigram_phrases.vocab.keys())

4310706

Prenons une clé au hasard :

In [54]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [55]:
bigram_phrases.vocab[key_]

42

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [56]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [57]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [58]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [60]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [61]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races', 'indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 

## Entrainement d'un modèle Word2Vec sur ce corpus

In [62]:
# Liste de combinaisons de paramètres à tester
params_list = [
    {"vector_size": 50, "window": 5, "min_count": 5},
    {"vector_size": 100, "window": 10, "min_count": 5},
    {"vector_size": 100, "window": 5, "min_count": 10},
]

models = {}

for i, params in enumerate(params_list):
    print(f"\n Entraînement du modèle {i+1} avec paramètres : {params}")
    model = Word2Vec(
        corpus,
        vector_size=params["vector_size"],
        window=params["window"],
        min_count=params["min_count"],
        workers=4,
        epochs=5
    )
    models[f"model_{i+1}"] = model
    model.save(f"../../data/word2vec_model_{i+1}.model")
    print(f" Modèle {i+1} sauvegardé !")



 Entraînement du modèle 1 avec paramètres : {'vector_size': 50, 'window': 5, 'min_count': 5}


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


 Modèle 1 sauvegardé !

 Entraînement du modèle 2 avec paramètres : {'vector_size': 100, 'window': 10, 'min_count': 5}


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


 Modèle 2 sauvegardé !

 Entraînement du modèle 3 avec paramètres : {'vector_size': 100, 'window': 5, 'min_count': 10}


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


 Modèle 3 sauvegardé !


## Exploration et comparaison des modèles

In [63]:
# Charger et explorer les modèles entraînés
for name, model in models.items():
    print(f"\n=== Exploration de {name} ===")

    # 3 exemples de similarité
    pairs = [("ministre", "roi"), ("homme", "femme"), ("paris", "londres")]
    for w1, w2 in pairs:
        try:
            sim = model.wv.similarity(w1, w2)
            print(f"Similarité({w1}, {w2}) = {sim:.3f}")
        except KeyError:
            print(f"{w1} ou {w2} absent du vocabulaire.")

    # 3 exemples de recherche de mots proches
    terms = ["ministre", "guerre", "belgique"]
    for t in terms:
        try:
            print(f"\nMots les plus proches de '{t}' :")
            for word, score in model.wv.most_similar(t, topn=5):
                print(f"  {word:20s} {score:.3f}")
        except KeyError:
            print(f"{t} absent du vocabulaire.")



=== Exploration de model_1 ===
Similarité(ministre, roi) = 0.510
Similarité(homme, femme) = 0.636
Similarité(paris, londres) = 0.848

Mots les plus proches de 'ministre' :
  secretaire           0.871
  president            0.840
  vice_-_president     0.809
  conseiller           0.802
  membre               0.795

Mots les plus proches de 'guerre' :
  la_defense           0.825
  la_guerre            0.807
  la_compagnie         0.799
  pekin                0.796
  retraite             0.792

Mots les plus proches de 'belgique' :
  la_societe           0.786
  suisse               0.769
  medecine             0.756
  nationale            0.747
  la_banque            0.739

=== Exploration de model_2 ===
Similarité(ministre, roi) = 0.464
Similarité(homme, femme) = 0.560
Similarité(paris, londres) = 0.799

Mots les plus proches de 'ministre' :
  secretaire           0.836
  conseiller           0.800
  president            0.796
  au_ministere         0.795
  ancien_ministre      0.78

#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [64]:
outfile = f"../../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [65]:
model = Word2Vec.load("../../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [66]:
model.wv["ministre"]
#moins c'est moins proche, plus c'est plur proche

array([ 1.1520694 ,  1.3869715 , -0.15809694,  0.09969696,  0.18600838,
        2.008419  ,  0.12284078, -3.0948699 , -0.31194934,  2.066266  ,
        0.43285382, -0.41917095, -0.8906473 ,  0.7589804 , -1.5893991 ,
        0.9553726 ,  1.5188748 ,  1.4067056 , -1.9166785 , -1.457321  ,
       -1.6248204 ,  0.02744159, -0.4742895 , -0.9052875 , -1.3184161 ,
        0.42229828, -0.4699981 ,  0.9755401 , -0.2022055 ,  1.7370551 ,
       -1.4279257 ,  0.5070898 ,  0.7569485 , -2.9274695 ,  0.3523946 ,
        1.8302702 ,  0.9074599 , -0.65073794, -1.0585675 ,  1.5529982 ,
        1.1959634 , -0.8633325 ,  2.0876932 ,  0.0814878 , -0.6044085 ,
        1.071245  ,  2.5926504 ,  0.36550343,  0.41242772, -0.67482823,
        1.219026  , -1.1494145 , -0.6225612 ,  1.1337495 , -1.5790877 ,
        0.15790279, -0.26247528,  0.87242895, -0.08747929, -0.42426404,
        0.6314056 ,  2.0613444 ,  0.2633526 ,  0.6484189 , -0.16809794,
       -0.42835483,  0.19014464,  3.3267062 , -1.0055785 , -1.30

### Calculer la similarité entre deux termes

In [67]:
model.wv.similarity("ministre", "roi")

np.float32(0.44898206)

### Chercher les mots les plus proches d'un terme donné

In [68]:
model.wv.most_similar("ministre", topn=10)

[('secretaire', 0.8031487464904785),
 ('membre', 0.7720927000045776),
 ('au_ministere', 0.769106924533844),
 ('vice_-_president', 0.766613245010376),
 ('president', 0.7633225917816162),
 ('depute', 0.7568985223770142),
 ('en_remplacement', 0.7503179311752319),
 ('secretaire_general', 0.7442861199378967),
 ('president_du_conseil', 0.742365300655365),
 ('bourgmestre', 0.7382998466491699)]

### Faire des recherches complexes à travers l'espace vectoriel

In [69]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))
#on retire le concept belgique aux concept paris et londre, 

[('berlin', 0.6801949739456177), ('marseille', 0.6304723024368286), ('ce_matin', 0.6293600797653198), ('nice', 0.6248372197151184), ('new_-_york', 0.6126053929328918), ('la_monnaie', 0.6120539307594299), ('dimanche', 0.5741748809814453), ('geneve', 0.5711570978164673), ('toulon', 0.5704194903373718), ('hier_soir', 0.5691279768943787)]
